In [85]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/ABBANK.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)   VOLUME
0  439 2018-05-08       ABBANK  11.8  ...  11.8    140       2.675  224,757
1  438 2018-06-08       ABBANK  12.0  ...  11.9    117       1.872  158,467
2  437 2018-07-08       ABBANK  12.0  ...  11.9    139       2.222  186,217
3  436 2018-08-08       ABBANK  12.3  ...  12.0    216       6.382  525,104
4  435 2018-09-08       ABBANK  12.5  ...  12.2    466      11.309  906,224

[5 rows x 12 columns]

In [86]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)   VOLUME
36  403 2018-01-10       ABBANK  12.6  ...  12.7    128       4.411  349,439
59  380 2018-01-11       ABBANK  11.3  ...  11.6    111       4.567  395,514
37  402 2018-02-10       ABBANK  12.4  ...  12.6    159       6.574  526,280
79  360 2018-02-12       ABBANK  11.4  ...  11.6    234       8.858  768,778
16  423 2018-03-09       ABBANK  13.5  ...  13.8    236       4.782  354,776

[5 rows x 12 columns]

In [87]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE  LTP*  HIGH  ...   YCP  TRADE  VALUE (mn)   VOLUME
DATE                                      ...                                  
2018-01-10  403       ABBANK  12.6  12.8  ...  12.7    128       4.411  349,439
2018-01-11  380       ABBANK  11.3  11.7  ...  11.6    111       4.567  395,514
2018-02-10  402       ABBANK  12.4  12.6  ...  12.6    159       6.574  526,280

[3 rows x 11 columns]

In [88]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [89]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [90]:
df['volatility'] = df['volatility']*100

In [91]:
import matplotlib.pyplot as plt 

In [92]:
df=df.reset_index()['volatility']

In [93]:
df.shape

(439,)

In [94]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [95]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [96]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [97]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [98]:

training_size,test_size

(307, 132)

In [99]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression

In [100]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [25]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [26]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [27]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [28]:
model4.fit(X_train, y_train)

[07:51:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [29]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [30]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [31]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [32]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [33]:
ytest

array([0.27730889, 0.2620578 , 0.26192661, 0.23766964, 0.23536174,
       0.23536174, 0.23536174, 0.23567004, 0.23567004, 0.23591855,
       0.23591855, 0.23591855, 0.22955837, 0.23324035, 0.23843065,
       0.25545461, 0.26651643, 0.22825789, 0.16835606, 0.19526147,
       0.19705091, 0.21883009, 0.21877757, 0.21948083, 0.22794534,
       0.26804589, 0.2871256 , 0.28892079, 0.32832192, 0.32832192,
       0.34793011])

In [34]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [35]:
Y_test =scaler.inverse_transform(Y_test)

In [36]:
Y_test

array([[5.77833297, 5.4605433 , 5.45780979, 4.95236307, 4.90427289,
        4.90427289, 4.90427289, 4.91069709, 4.91069709, 4.9158754 ,
        4.9158754 , 4.9158754 , 4.78334713, 4.86006931, 4.96822042,
        5.32295157, 5.55344854, 4.75624882, 3.50806423, 4.0686967 ,
        4.10598373, 4.5598002 , 4.5587057 , 4.57335973, 4.74973621,
        5.58531825, 5.98288539, 6.02029215, 6.84130014, 6.84130014,
        7.24987936]])

In [37]:
test_predict1

array([0.43066617, 0.41556584, 0.4130364 , 0.41627781, 0.41934969,
       0.42222921, 0.42455167, 0.43160211, 0.4367553 , 0.44305452,
       0.44492002, 0.44696772, 0.45565323, 0.4563069 , 0.45625241,
       0.47707277, 0.47630619, 0.45399727, 0.45486961, 0.47606717,
       0.47258077, 0.47160962, 0.46780679, 0.46607515, 0.46493445,
       0.46348708, 0.46142933, 0.47240988, 0.47059409, 0.46668288,
       0.46510523])

In [39]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict1.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict1.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict1.size))



In [40]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [41]:
test_predict1

array([[8.97386486, 8.65921679, 8.6065104 , 8.67405216, 8.73806152,
        8.7980626 , 8.84645598, 8.99336718, 9.1007452 , 9.23200303,
        9.27087493, 9.31354323, 9.4945246 , 9.50814527, 9.50700974,
        9.94084731, 9.92487397, 9.46001905, 9.47819624, 9.91989327,
        9.84724672, 9.82701066, 9.74777042, 9.71168803, 9.68791905,
        9.65775978, 9.61488211, 9.8436857 , 9.80584984, 9.72435139,
        9.69147759]])

In [42]:
test_predict2

array([[10.40560617, 10.39020764, 10.39403947, 10.42535222, 10.49563035,
        10.50150464, 10.51012815, 10.53103666, 10.55272845, 10.58051469,
        10.6080172 , 10.7206695 , 11.78404751, 11.78075946, 11.77484872,
        11.7809313 , 11.76113049, 11.75051123, 11.80358703, 11.6418457 ,
        11.63855363, 11.54560302, 11.67859044, 11.65252099, 11.5278222 ,
        11.42531382, 11.28730278, 11.5278222 , 11.50853516, 11.4218151 ,
        11.76195683]])

In [43]:
test_predict3

array([[11.61912094, 11.78409756, 11.49146099, 11.85813219, 11.90705942,
        11.97533069, 12.12992193, 12.22015567, 12.27119999, 12.33041232,
        12.33725649, 12.68542311, 12.12897563, 12.11886115, 12.10870052,
        12.47192985, 12.16376102, 11.80511918, 12.13023178, 12.0239808 ,
        12.05785916, 12.04202232, 11.809655  , 11.79612873, 11.79280053,
        11.96848025, 11.6769455 , 12.07916127, 12.08468391, 11.38734758,
        11.79626284]])

In [44]:
test_predict4

array([[10.85647  , 10.826629 , 11.100321 , 11.377728 , 11.326037 ,
        11.3700695, 11.406154 , 11.615579 , 11.464229 , 11.464229 ,
        11.464229 , 12.628555 , 13.126772 , 13.126772 , 13.126772 ,
        13.55475  , 13.497537 , 13.222274 , 13.229899 , 12.617921 ,
        12.616557 , 12.616557 , 12.479215 , 12.479215 , 12.479215 ,
        12.479215 , 12.479215 , 12.23064  , 12.23064  , 12.008181 ,
        13.082889 ]], dtype=float32)

In [45]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

4.338438152190123

In [46]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

6.12296039141622

In [47]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

6.90837336414254

In [48]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

7.186958073721025

MAE

In [50]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 4.250949


In [52]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 6.043498


In [51]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 6.846193


In [53]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 7.089160


MAPE